In [1]:
from numpy import genfromtxt
import tensorflow as tf
import numpy as np

data1 = genfromtxt('/users/facsupport/asharma/Data/batches/31/OH/one.csv',delimiter=',')
data2 = genfromtxt('/users/facsupport/asharma/Data/batches/31/OH/two.csv',delimiter=',')
data3 = genfromtxt('/users/facsupport/asharma/Data/batches/31/OH/three.csv',delimiter=',')

In [6]:
print(data1.shape)
print(data2.shape)
print(data3.shape)
train_set = np.concatenate((data1,data2),axis=0)
test_set = data3

(28187, 61)
(17048, 61)
(17559, 61)


In [8]:
def make_traindata(set):
    examples = np.concatenate((set[:,:30],set[:,33:]),axis=1) #Concatenate time series and OH encoding
    labels = set[:,30:31]

    return tf.data.Dataset.from_tensor_slices((examples,labels))


In [34]:
train_dataset = make_traindata(train_set)

BATCH_SIZE = 8

train_dataset = train_dataset.batch(BATCH_SIZE)

tf.keras.backend.set_floatx('float64')

In [35]:
train_dataset.take(1)


<TakeDataset shapes: ((None, 58), (None, 1)), types: (tf.float64, tf.float64)>

In [36]:
loss_object = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.Adam(2e-5, beta_1=0.5)

def testModel(testSet, model):
    absError = 0
    length = 0
    for entry in testSet:
        
        seq = np.concatenate((entry[:30],entry[33:]),axis=0)
        seq = np.array([[seq]])
        seq = np.transpose(seq, axes=[0, 2, 1])

        label = entry[30:31]
        mean = entry[31:32]
        std = entry[32:33]

        prediction = model(seq)
        prediction = (prediction*std)+mean
        label = (label*std)+mean
        print("Prediction: "+str(prediction)+" True Value: "+str(label))
        if np.isnan(abs(prediction-label)):
            continue
        length += 1
        absError += abs(prediction-label)
    print("Mean Absolute Error: "+str(absError/length))    
    


In [37]:
def train_epoch(model):
 i = 0
 nanCount = 0
 for example, label in train_dataset:
     example = np.expand_dims(example,2)
     label = np.expand_dims(label,2)
     with tf.GradientTape() as tape:
         prediction = model(example)
         loss = loss_object(prediction,label)
         if loss.dtype == tf.int64:
             continue

         
         gradients = tape.gradient(loss,model.trainable_variables)
         if np.isnan(gradients[0].numpy())[0][0]:
            nanCount+=1 
            continue
         i += 1
         if i%1000 == 1:    
            print("LOSS: "+str(loss))
         optimizer.apply_gradients(zip(gradients,model.trainable_variables))
 print("NAN Count: "+str(nanCount)+" Successfully trained batches: "+str(i))


In [38]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, input_dim = 58),
    tf.keras.layers.Dense(32),
    tf.keras.layers.Dense(1)
])

In [39]:
for i in range(10):
    print("EPOCH: "+str(i))
    train_epoch(model)
    

EPOCH: 0
LOSS: tf.Tensor(1.2838565111160278, shape=(), dtype=float64)
LOSS: tf.Tensor(0.8658167123794556, shape=(), dtype=float64)
LOSS: tf.Tensor(0.7205873727798462, shape=(), dtype=float64)
LOSS: tf.Tensor(0.7944233417510986, shape=(), dtype=float64)
LOSS: tf.Tensor(0.6088616847991943, shape=(), dtype=float64)
LOSS: tf.Tensor(0.323394238948822, shape=(), dtype=float64)
NAN Count: 260 Successly trained batches: 5395
EPOCH: 1
LOSS: tf.Tensor(0.857864499092102, shape=(), dtype=float64)
LOSS: tf.Tensor(0.6226292252540588, shape=(), dtype=float64)
LOSS: tf.Tensor(0.633002519607544, shape=(), dtype=float64)
LOSS: tf.Tensor(0.7169365882873535, shape=(), dtype=float64)
LOSS: tf.Tensor(0.6022996306419373, shape=(), dtype=float64)
LOSS: tf.Tensor(0.3293924331665039, shape=(), dtype=float64)
NAN Count: 260 Successly trained batches: 5395
EPOCH: 2
LOSS: tf.Tensor(0.8746129870414734, shape=(), dtype=float64)
LOSS: tf.Tensor(0.5518472790718079, shape=(), dtype=float64)
LOSS: tf.Tensor(0.6246602535

In [40]:
testModel(test_set[0:10],model)


Prediction: tf.Tensor([[8.75735677]], shape=(1, 1), dtype=float64) True Value: [8.]
Prediction: tf.Tensor([[8.98448375]], shape=(1, 1), dtype=float64) True Value: [7.75]
Prediction: tf.Tensor([[7.83813437]], shape=(1, 1), dtype=float64) True Value: [7.5]
Prediction: tf.Tensor([[8.26303863]], shape=(1, 1), dtype=float64) True Value: [7.75]
Prediction: tf.Tensor([[8.20121309]], shape=(1, 1), dtype=float64) True Value: [7.25]
Prediction: tf.Tensor([[5.47220158]], shape=(1, 1), dtype=float64) True Value: [0.]
Prediction: tf.Tensor([[7.51059971]], shape=(1, 1), dtype=float64) True Value: [7.5]
Prediction: tf.Tensor([[7.19941495]], shape=(1, 1), dtype=float64) True Value: [6.82]
Prediction: tf.Tensor([[8.83370921]], shape=(1, 1), dtype=float64) True Value: [8.]
Prediction: tf.Tensor([[9.22714197]], shape=(1, 1), dtype=float64) True Value: [7.75]
Mean Absolute Error: tf.Tensor([[1.1967294]], shape=(1, 1), dtype=float64)


In [41]:
testModel(test_set[3000:3021],model)

Prediction: tf.Tensor([[4.7332688]], shape=(1, 1), dtype=float64) True Value: [7.8]
Prediction: tf.Tensor([[0.99563956]], shape=(1, 1), dtype=float64) True Value: [1.83]
Prediction: tf.Tensor([[0.86601183]], shape=(1, 1), dtype=float64) True Value: [5.55111512e-17]
Prediction: tf.Tensor([[4.67630202]], shape=(1, 1), dtype=float64) True Value: [8.]
Prediction: tf.Tensor([[2.59478052]], shape=(1, 1), dtype=float64) True Value: [0.]
Prediction: tf.Tensor([[2.00914667]], shape=(1, 1), dtype=float64) True Value: [0.]
Prediction: tf.Tensor([[0.51059668]], shape=(1, 1), dtype=float64) True Value: [0.83]
Prediction: tf.Tensor([[2.58377516]], shape=(1, 1), dtype=float64) True Value: [0.]
Prediction: tf.Tensor([[4.36353482]], shape=(1, 1), dtype=float64) True Value: [8.]
Prediction: tf.Tensor([[4.78812055]], shape=(1, 1), dtype=float64) True Value: [9.5]
Prediction: tf.Tensor([[3.31495307]], shape=(1, 1), dtype=float64) True Value: [5.]
Prediction: tf.Tensor([[1.43262729]], shape=(1, 1), dtype=f

In [42]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(256, input_dim = 58,activation='relu'),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(64),
    tf.keras.layers.Dense(1)
])

for i in range(10):
    print("EPOCH: "+str(i))
    train_epoch(model)
    

EPOCH: 0
LOSS: tf.Tensor(1.0954170227050781, shape=(), dtype=float64)
LOSS: tf.Tensor(0.6700652837753296, shape=(), dtype=float64)
LOSS: tf.Tensor(0.6692762970924377, shape=(), dtype=float64)
LOSS: tf.Tensor(0.6526451110839844, shape=(), dtype=float64)
LOSS: tf.Tensor(0.5897113084793091, shape=(), dtype=float64)
LOSS: tf.Tensor(0.3126004934310913, shape=(), dtype=float64)
NAN Count: 260 Successly trained batches: 5395
EPOCH: 1
LOSS: tf.Tensor(0.8880847692489624, shape=(), dtype=float64)
LOSS: tf.Tensor(0.38249534368515015, shape=(), dtype=float64)
LOSS: tf.Tensor(0.6428633332252502, shape=(), dtype=float64)
LOSS: tf.Tensor(0.624163806438446, shape=(), dtype=float64)
LOSS: tf.Tensor(0.5882889032363892, shape=(), dtype=float64)
LOSS: tf.Tensor(0.30569177865982056, shape=(), dtype=float64)
NAN Count: 260 Successly trained batches: 5395
EPOCH: 2
LOSS: tf.Tensor(0.9453437328338623, shape=(), dtype=float64)
LOSS: tf.Tensor(0.27143213152885437, shape=(), dtype=float64)
LOSS: tf.Tensor(0.63305

In [43]:
testModel(test_set[50:60],model)

Prediction: tf.Tensor([[4.92573785]], shape=(1, 1), dtype=float64) True Value: [6.]
Prediction: tf.Tensor([[1.14519119]], shape=(1, 1), dtype=float64) True Value: [-5.55111512e-17]
Prediction: tf.Tensor([[4.64413016]], shape=(1, 1), dtype=float64) True Value: [7.5]
Prediction: tf.Tensor([[2.0134711]], shape=(1, 1), dtype=float64) True Value: [3.5]
Prediction: tf.Tensor([[1.98581242]], shape=(1, 1), dtype=float64) True Value: [0.]
Prediction: tf.Tensor([[2.15978322]], shape=(1, 1), dtype=float64) True Value: [2.]
Prediction: tf.Tensor([[5.46744123]], shape=(1, 1), dtype=float64) True Value: [8.]
Prediction: tf.Tensor([[4.45836357]], shape=(1, 1), dtype=float64) True Value: [7.5]
Prediction: tf.Tensor([[3.48950905]], shape=(1, 1), dtype=float64) True Value: [0.]
Prediction: tf.Tensor([[2.13621282]], shape=(1, 1), dtype=float64) True Value: [0.]
Mean Absolute Error: tf.Tensor([[1.99073648]], shape=(1, 1), dtype=float64)


In [44]:
testModel(test_set[8000:8010],model)

Prediction: tf.Tensor([[8.41947501]], shape=(1, 1), dtype=float64) True Value: [12.]
Prediction: tf.Tensor([[5.80485448]], shape=(1, 1), dtype=float64) True Value: [6.5]
Prediction: tf.Tensor([[9.52972942]], shape=(1, 1), dtype=float64) True Value: [6.4]
Prediction: tf.Tensor([[6.6188204]], shape=(1, 1), dtype=float64) True Value: [7.5]
Prediction: tf.Tensor([[3.73537961]], shape=(1, 1), dtype=float64) True Value: [6.]
Prediction: tf.Tensor([[5.60549868]], shape=(1, 1), dtype=float64) True Value: [7.25]
Prediction: tf.Tensor([[5.95246408]], shape=(1, 1), dtype=float64) True Value: [7.25]
Prediction: tf.Tensor([[6.18860879]], shape=(1, 1), dtype=float64) True Value: [10.6]
Prediction: tf.Tensor([[4.84722307]], shape=(1, 1), dtype=float64) True Value: [7.5]
Prediction: tf.Tensor([[8.8576453]], shape=(1, 1), dtype=float64) True Value: [11.5]
Mean Absolute Error: tf.Tensor([[2.319976]], shape=(1, 1), dtype=float64)
